In [ ]:
import tkinter as tk
from tkinter import ttk
from datetime import datetime, timedelta
import requests
from tkinter import messagebox
import pandas as pd

class ProposicoesGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Consulta de Proposições")
        
        input_frame = ttk.Frame(self.root, padding="20")
        input_frame.pack(padx=20, pady=20)

        ttk.Label(input_frame, text="Palavras-chave:").grid(row=0, column=0, sticky="w")
        self.keywords_entry = ttk.Entry(input_frame, width=40)
        self.keywords_entry.grid(row=0, column=1, padx=10, pady=5)
        
        ttk.Label(input_frame, text="Data Início (YYYY-MM-DD):").grid(row=1, column=0, sticky="w")
        self.data_inicio_entry = ttk.Entry(input_frame, width=15)
        self.data_inicio_entry.grid(row=1, column=1, padx=10, pady=5)
        
        ttk.Label(input_frame, text="Data Fim (YYYY-MM-DD):").grid(row=2, column=0, sticky="w")
        self.data_fim_entry = ttk.Entry(input_frame, width=15)
        self.data_fim_entry.grid(row=2, column=1, padx=10, pady=5)
        
        consultar_button = ttk.Button(input_frame, text="Consultar", command=self.consultar_proposicoes)
        consultar_button.grid(row=3, columnspan=2, pady=10)
        
        self.projetos_lista = []
        
    def consultar_proposicoes(self):
        base_url = "https://dadosabertos.camara.leg.br/api/v2"
        endpoint = "/proposicoes"
        
        keywords = self.keywords_entry.get()
        data_inicio = self.data_inicio_entry.get()
        data_fim = self.data_fim_entry.get()
        
        params = {
            "ano": [2024],
            "keywords": keywords,
            "dataInicio": data_inicio,
            "dataFim": data_fim,
            "ordenarPor": "id"
        }

        headers = {
            "Accept": "application/json"
        }
        
        try:
            response = requests.get(base_url + endpoint, params=params, headers=headers)
            if response.status_code == 200:
                dados = response.json()
                proposicoes = dados['dados']
                for proposicao in proposicoes:
                    url_procura_id = f"https://dadosabertos.camara.leg.br/api/v2/proposicoes/{proposicao['id']}"
                    projeto_id = requests.get(url_procura_id)
                    resposta_projeto = projeto_id.json()
                    projeto_dict = {
                        "ID": resposta_projeto['dados']['id'],
                        "Tipo": resposta_projeto['dados']['siglaTipo'],
                        "Ano": resposta_projeto['dados']['ano'],
                        "Ementa": resposta_projeto['dados']['ementa'],
                        "Data Inicio": resposta_projeto['dados']['dataApresentacao'],
                        "Ultma Alteracao": resposta_projeto['dados']['statusProposicao']['dataHora'],
                        "Palavras Chaves": resposta_projeto['dados']['keywords'],
                        "Link": f"https://www.camara.leg.br/proposicoesWeb/fichadetramitacao?idProposicao={resposta_projeto['dados']['id']}"
                    }
                    self.projetos_lista.append(projeto_dict)
                
                relatorio_df = pd.DataFrame(self.projetos_lista)
                print(relatorio_df)
                relatorio_df.to_excel("Projetos-Novos.xlsx", index=False)
                messagebox.showinfo("Sucesso", "Consulta realizada com sucesso. Resultados salvos em Projetos-Novos.xlsx.")
            else:
                messagebox.showerror("Erro", f"Código de status da requisição: {response.status_code}")
        
        except Exception as e:
            messagebox.showerror("Erro", f"Ocorreu um erro ao realizar a consulta: {str(e)}")

def main():
    root = tk.Tk()
    app = ProposicoesGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()

         ID Tipo   Ano                                             Ementa  \
0   2416990   PL  2024  Dispõe sobre o fornecimento temporário de ener...   
1   2417060   PL  2024  Altera o art. 155 do Decreto-Lei nº 2.848, de ...   
2   2417293   PL  2024  Dispõe sobre a obrigatoriedade da disponibiliz...   
3   2418078   PL  2024  Dispõe sobre comercialização de créditos de en...   
4   2418087   PL  2024  Estabelece percentual mínimo de participação d...   
5   2418094   PL  2024  Altera a Lei nº 9.427, de 26 de dezembro de 19...   
6   2418717   PL  2024  Dispõe sobre os indicadores e metas para avali...   
7   2418718   PL  2024  Dispõe sobre a participação dos estados da fed...   
8   2418719   PL  2024  Dispõe sobre a disponibilidade e transparência...   
9   2418793  PLP  2024  Altera o art. 32 do Código Tributário Nacional...   
10  2418961   PL  2024  Dispõe sobre cobrança de compensação financeir...   
11  2419272   PL  2024  Altera a Lei nº 12.212, de 20 de janeiro de 20...   